## Valmistautuminen

Jos käytössäsi on Windows-kone, saatat joutua tekemään lisäasennuksia jotta saat [fastText-kirjaston asennettua](https://medium.com/@oleg.tarasov/building-fasttext-python-wrapper-from-source-under-windows-68e693a68cbb). 
Toinen vaihtoehto on toteuttaa tämä harjoitus [Google Colabissa](https://colab.research.google.com/). 


1. Asennettava: <code>fasttext</code>, <code>nltk</code>, <code>spacy</code> 
1. Käynnistä kernel uudelleen
1. Jos virheet jatkuvat, asenna:  <code>spacy-transformers</code> 
1. Käynnistä kernel uudelleen
1. <code>python -m spacy download fi_core_news_sm</code>



In [ ]:
#!pip install fasttext nltk spacy
#!pip install spacy-transformers

In [2]:
!python -m spacy download fi_core_news_sm

     ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
     - -------------------------------------- 0.5/14.3 MB 3.4 MB/s eta 0:00:05
     -- ------------------------------------- 1.0/14.3 MB 2.8 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/14.3 MB 2.1 MB/s eta 0:00:07
     ---- ----------------------------------- 1.6/14.3 MB 1.9 MB/s eta 0:00:07
     ----- ---------------------------------- 2.1/14.3 MB 2.0 MB/s eta 0:00:07
     ------ --------------------------------- 2.4/14.3 MB 2.1 MB/s eta 0:00:06
     ------- -------------------------------- 2.6/14.3 MB 1.8 MB/s eta 0:00:07
     -------- ------------------------------- 3.1/14.3 MB 1.9 MB/s eta 0:00:06
     ---------- ----------------------------- 3.9/14.3 MB 2.1 MB/s eta 0:00:06
     ------------ --------------------------- 4.5/14.3 MB 2.1 MB/s eta 0:00:05
     ------------- -------------------------- 5.0/14.3 MB 2.2 MB/s eta 0:00:05
     ---------------- ----------------------- 5.8/14.3 MB 2

Following Teemu's presentation earlier this week, the trained FastText classifier was downloaded using the following lines:
<code>
    ft_classifier.quantize(input=input, retrain=True)
    ft_classifier.save_model("ft_classifier.ftz")
</code>

In [1]:
#import fasttext
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
# import libvoikko #perusmutoistamiseen käytetty kirjasto
import spacy

In [ ]:
# !python -m spacy download fi_core_news_sm

In [3]:
nlp = spacy.load("fi_core_news_sm")
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Valtteri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
print(stopwords.words('finnish'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Valtteri/nltk_data'
    - 'c:\\Users\\Valtteri\\anaconda3\\nltk_data'
    - 'c:\\Users\\Valtteri\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\Valtteri\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Valtteri\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# Load tweets datasets
# Note: May need to install openpyxl library for the following line to work
# pip install openpyxl
df = pd.read_excel('shared_data/sample_texts.xlsx')

In [ ]:
df.head()

In [ ]:
tweets = df[['text_nousernames', 'eduskunta-vkk']]

In [ ]:
tweets.head()

In [ ]:
# Methods imported from the original Notebook
def preprocess_word(word, stopwords):
    word = re.sub("[^A-Za-z0-9ÄäÖö-]+", "", word)
    word = re.sub("-", " ", word)
    word = word.strip().lower()
    if len(word) < 2:
        return ""
    if type(word) != None:
        if word in stopwords:
            return ""
        return word

def preprocess_sent(sent, stopwords):
    if sent != "":
        # lemmatize words using spacy
        doc = nlp(sent)
        sent = " ".join([token.lemma_ for token in doc])
        
        words = sent.strip().split(" ")
        final_sent = " ".join([preprocess_word(word, stopwords) for word in words if type(word) != None])
        return final_sent

def preprocess_row(teksti, stopwords):
    if type(teksti) == str:
        sents = teksti.strip().split(".")
        sents = filter(None,sents)
        teksti = ".".join([preprocess_sent(sent, stopwords) for sent in sents if type(sent) != None])
        return teksti

In [ ]:
tweets.head()

In [ ]:
#Apufunktio, jolla voidaan luoda fastText-yhteensopiva opetus- ja tekstitiedosto

def preprocess_ft_df(df):
    df["label"] = ["__label__" + "_".join(str(x).split(" ")) for x in df["eduskunta-vkk"].values]
    df["preprocessed"] = [preprocess_row(sentence, stopwords.words('finnish')) for sentence in df["text_nousernames"]]
    df = df.drop(["eduskunta-vkk", "text_nousernames"], axis = 1)
    return df

In [ ]:
new_df = preprocess_ft_df(tweets)

In [ ]:
new_df.head()

In [ ]:

model = fasttext.load_model("model_filename.ftz")

In [ ]:
model.predict('Hiilen sidonta vaatii toimenpiteitä metsäpolitiikassa')[0][0]


In [ ]:
# TODO: Produce predictions for all tweets. Measure the overall accuracy (correct predictions / mistakes)

